In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf

In [2]:
# home_df = pd.read_csv('../Pt3_Data_processing/home_structured_ewm16.csv')
# away_df = pd.read_csv('../Pt3_Data_processing/away_structured_ewm16.csv')

df = pd.read_csv('../Pt3_Data_processing/home_structured_ewm20.csv')
df.sort_values(by='h_pg_date', inplace=True)

# Turning IDS into categorical variables
for col in df:
    if 'id' in col:
        df[col]=df[col].astype('category')
    
# Turning age into integers        
for col in [col for col in df if 'age' in col]:
    df[col] = df[col].apply(lambda x: int(x.replace(' days 00:00:00.000000000','')))
    
df.drop_duplicates(inplace=True)

name_id_remove = [col for col in df if any(c in col for c in ['_player_id','_name', '_team'])]

modelling_columns = ['h_pg_outcome', 'h_pg_difference', 'h_pg_type', 'h_pg_location'] + \
[col for col in df.loc[:,'h_pg_age_days':] if \
                     col not in name_id_remove]

X = df[modelling_columns].copy()
# X.fillna(value=0, inplace=True)

print('COVID\n',X['h_pg_outcome'][X['a_pg_date']>'2020-03-30'].value_counts(normalize=True))
print('\n2018-\n',X['h_pg_outcome'][X['a_pg_date']>'2018-08-30'].value_counts(normalize=True))
print('\n-2018\n',X['h_pg_outcome'][X['a_pg_date']<'2018-08-30'].value_counts(normalize=True))
print('\n2019-\n',X['h_pg_outcome'][X['a_pg_date']>'2019-08-30'].value_counts(normalize=True))
print('\n-2019\n',X['h_pg_outcome'][X['a_pg_date']<'2019-08-30'].value_counts(normalize=True))

X[X['a_pg_date']>'2017-08-30'].shape[0], X[X['a_pg_date']<'2017-08-30'].shape[0]

# REMOVING FIRST YEAR OF GAMES AND POST-COVID 
# (GAMES NOT PLAYED ON HOME COURTS, DATA CONSTRUCTION WOULD CAUSE ISSUES)
X.drop(X[X['a_pg_date']<'2001-08-30'].index, inplace=True)
X.drop(X[X['a_pg_date']>'2020-03-30'].index, inplace=True)

print('\nBASELINE FOR TOTAL DATASET :\n',X['h_pg_outcome'].value_counts(normalize=True))

print('\nDF SHAPE :\n',X.shape)

for i in range(16):
#     PREFIX FOR NEW COLUMN NAMES
    prefix = [col for col in X if 'fgm' in col][i][:4]
#     FG%
    X[f'{prefix}_fgpct'] = X[[col for col in X if 'fgm' in col][i]] / \
    X[[col for col in X if 'fga' in col][i]]
#     FG3%
    X[f'{prefix[:-4]}_fg3pct'] = X[[col for col in X if 'fg3m' in col][i]] / \
    X[[col for col in X if 'fg3a' in col][i]]
#     FT%
    X[f'{prefix[:-4]}_ftpct'] = X[[col for col in X if 'ftm' in col][i]] / \
    X[[col for col in X if 'fta' in col][i]]
#     JOHN HOLLINGER GAMESCORE
    X[f'{prefix}_gs'] = X[f'{prefix}_ewm_pts'] + .4*X[[col for col in X if 'fgm' in col][i]] - \
    .7*X[[col for col in X if 'fga' in col][i]] - .4*(X[[col for col in X if 'fta' in col][i]] - \
                                                     X[[col for col in X if 'ftm' in col][i]]) + \
    .7*X[[col for col in X if 'oreb' in col][i]] + .3*X[[col for col in X if 'dreb' in col][i]] + \
    X[[col for col in X if '_stl' in col][i]] + .7*X[[col for col in X if 'ewm_ast' in col][i]] + \
    .7*X[[col for col in X if 'ewm_blk' in col][i]] - .4*X[[col for col in X if 'ewm_pf' in col][i]] - \
    X[[col for col in X if 'ewm_to' in col][i]]
    

COVID
 1    0.533333
0    0.466667
Name: h_pg_outcome, dtype: float64

2018-
 1    0.571667
0    0.428333
Name: h_pg_outcome, dtype: float64

-2018
 1    0.600805
0    0.399195
Name: h_pg_outcome, dtype: float64

2019-
 1    0.549639
0    0.450361
Name: h_pg_outcome, dtype: float64

-2019
 1    0.600271
0    0.399729
Name: h_pg_outcome, dtype: float64

BASELINE FOR TOTAL DATASET :
 1    0.597252
0    0.402748
Name: h_pg_outcome, dtype: float64

DF SHAPE :
 (23002, 539)


In [3]:
X.drop(columns=['a_pg_season_id', 'a_pg_location', 'a_pg_type',
       'a_pg_outcome', 'a_pg_difference', 'a_pg_game_id'], inplace=True)

X.fillna(0, inplace=True)


X_train = X[X['a_pg_date']<'2017-08-30'].copy()
X_test = X[X['a_pg_date']>'2017-08-30'].copy()

X_train.drop(columns='a_pg_date', inplace=True)
X_test.drop(columns='a_pg_date', inplace=True)



yC_train = X_train.pop('h_pg_outcome')
yC_test = X_test.pop('h_pg_outcome')

yR_train = X_train.pop('h_pg_difference')
yR_test = X_test.pop('h_pg_difference')

columns = X_train.columns

In [6]:
poly = PolynomialFeatures()
poly.fit(X_train)
X_train = pd.DataFrame(poly.transform(X_train),
                       columns=poly.get_feature_names([c for c in X_train if c!=0]))
X_test = pd.DataFrame(poly.transform(X_test),
                       columns=poly.get_feature_names([c for c in X_train if c!=0]))

power = PowerTransformer()
X_train = pd.DataFrame(power.fit_transform(X_train),
                       columns=X_train.columns)
X_test = pd.DataFrame(power.transform(X_test),
                      columns=X_train.columns)

NameError: name 'PowerTransformer' is not defined

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000, max_features='sqrt')

rfc_params = {'criterion' : ['gini','entropy']}

rfc_grid = GridSearchCV(rfc, rfc_params, n_jobs=2, verbose=3)

rfc_grid.fit(X_train, y_train)

print('Train R2 score : ', rfc_grid.score(X_train_pca, y_train_seas))
print('Test R2 score  : ', rfc_grid.score(X_test_pca, y_test_seas))
print('Mean CV score. : ', rfc_grid.best_score_)
print(rfc_grid.best_params_)

In [4]:
import numpy as np

In [ ]:
power = PowerTransformer()
X_train = pd.DataFrame(power.fit_transform(X_train),
                       columns=X_train.columns)
X_test = pd.DataFrame(power.transform(X_test),
                      columns=X_train.columns)

In [6]:
np.e

2.718281828459045